# Adaptive PDE discretizations on cartesian grids 
## Volume : Algorithmic tools
## Part : Automatic differentiation
## Chapter : Known bugs and incompatibilities

The techniques of automatic differentiation technique play an essential role in the notebooks presented in this repository. 

[**Summary**](Summary.ipynb) of volume Algorithmic tools, this series of notebooks.

[**Main summary**](../Summary.ipynb) of the Adaptive Grid Discretizations 
	book of notebooks, including the other volumes.

# Table of contents
  * [1 Matrix multiplication and inversion](#1-Matrix-multiplication-and-inversion)
  * [2. In place modifications and aliasing](#2.-In-place-modifications-and-aliasing)
    * [2.1 Aliasing of the AD information](#2.1-Aliasing-of-the-AD-information)
    * [2.2 Non writeable AD information](#2.2-Non-writeable-AD-information)



**Acknowledgement.** The experiments presented in these notebooks are part of ongoing research, 
some of it with PhD student Guillaume Bonnet, in co-direction with Frederic Bonnans.

Copyright Jean-Marie Mirebeau, University Paris-Sud, CNRS, University Paris-Saclay

## 0. Importing the required libraries

In [1]:
import sys; sys.path.insert(0,"..") # Allow importing agd from parent directory
#from Miscellaneous import TocTools; TocTools.displayTOC('ADBugs','Algo')

In [2]:
import numpy as np
import scipy.sparse.linalg

In [3]:
import agd.AutomaticDifferentiation as ad

In [4]:
def reload_packages():
    from Miscellaneous.rreload import rreload
    global ad
    ad, = rreload([ad],rootdir='..',verbose=True)

## 1 Matrix multiplication and inversion

Please use the `ad.apply_linear_mapping` and `ad.apply_linear_inverse` functions in combination with `np.dot`, or scipy solve functions.

In [5]:
v = ad.Dense.denseAD( np.random.standard_normal((4,)),np.random.standard_normal((4,4)))
m0 = np.random.standard_normal((4,4))
m1 = scipy.sparse.coo_matrix( ([1.,2.,3.,4.,5.],([0,2,1,2,3],[0,1,2,2,3]))).tocsr()

In [6]:
# Fails
#print("np.dot looses AD:",np.dot(m0,v))
#print("scipy '*' looses AD:",m1*v) 

In [7]:
print("np.dot with AD:\n",ad.apply_linear_mapping(m0,v))
print("scipy '*' with AD:\n",ad.apply_linear_mapping(m1,v))

np.dot with AD:
 denseAD([ 0.76160817  1.36598993  0.87729725 -0.88802633],
[[ 2.84727849 -3.66499607  3.30779637  0.6322657 ]
 [-4.75884078 -0.93638453  0.28790817 -3.17649471]
 [-1.17251102  1.15679684  0.53120487  0.78984788]
 [ 0.0469179   0.9245308  -2.6544424   0.09751143]])
scipy '*' with AD:
 denseAD([0.01837249 1.91715302 0.3678824  0.68035842],
[[ 2.60376579e-01 -2.17001037e+00 -4.16260160e-03 -1.05739462e+00]
 [ 6.97430852e-01  1.24771019e+00  8.10096012e-01  4.26756267e+00]
 [ 4.63927143e+00  2.62284769e+00 -3.16839045e-01  6.90181143e+00]
 [-6.85618459e+00  7.75139576e+00 -4.60481962e+00  7.07744189e-01]])


In [8]:
print("scipy solve with AD :\n",ad.apply_linear_inverse(scipy.sparse.linalg.spsolve,m1,v))

scipy solve with AD :
 denseAD([ 0.01837249  1.04896605 -0.36472027  0.02721434],
[[ 0.26037658 -2.17001037 -0.0041626  -1.05739462]
 [-1.12021607 -0.111793    0.60067169  0.30735115]
 [ 0.61822727  0.15987235 -0.23282784  0.20195465]
 [-0.27424738  0.31005583 -0.18419278  0.02830977]])


## 2. In place modifications and aliasing

The AD information often consists of very large arrays. In order to save time and memory, this information is not systematically copied and/or stored fully. It can take the form of a broadcasted array, or of an alias to another array. In that case a copy is necessary to enable modifications.

### 2.1 Aliasing of the AD information

When an operation leaves the AD information untouched, an alias is used. This can lead to bugs if in place modifications are used afterward.

In [9]:
x=ad.Dense.identity(constant=np.array([1.,2.]))
y=x+1 # Only affects the value, not the AD information

In [10]:
print("Values are distinct :", x.value is y.value)
print("AD information is shared :", y.coef is x.coef)

Values are distinct : False
AD information is shared : True


A modification of the aliased variable will impact the original one.

In [11]:
print(x[0])
y[0]*=2
print("Caution ! Shared AD information is affected :", x[0])

denseAD(1.0,[1. 0.])
Caution ! Shared AD information is affected : denseAD(1.0,[2. 0.])


Avoid this effect by making a copy.

In [12]:
x=ad.Dense.identity(constant=np.array([1.,2.]))
y=(x+1).copy()
print("AD information is distinct :", y.coef is x.coef)

AD information is distinct : False


Note that a similar effect arises with the `-` binary operator, but not with `*`or `/`. That is because the latter modify the AD information, which therefore must be copied anyway.

In [13]:
x=ad.Dense.identity(constant=np.array([1.,2.]))
print("AD information is shared :", (x-1).coef is x.coef)
print("AD information is distinct :", (x*2).coef is x.coef)
print("AD information is distinct :", (x/2).coef is x.coef)

AD information is shared : True
AD information is distinct : False
AD information is distinct : False


### 2.2 Non writeable AD information

When creating an dense AD variable, the coefficients may be non writeable (e.g. broadcasted) arrays.

In [14]:
x=ad.Dense.identity(constant=np.array([[1.,2.],[3.,4.]]),shape_bound=(2,))

In [15]:
x.coef.flags.writeable

False

In [16]:
# x+=1 # Fails because non-writeable

Make a copy to solve the issue.

In [17]:
y=x.copy()

In [18]:
y.coef.flags.writeable

True

In [19]:
y+=1